<a href="https://colab.research.google.com/github/nbPittigiani/crime_analysis_project/blob/main/cleaning_bank_mkt_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported.

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [157]:
import pandas as pd
import numpy as np

In [158]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [159]:
# Display column names
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [160]:
# Split dataset
client = df[['client_id', 'age', 'job', 'marital',
                'education', 'credit_default','mortgage']]
campaign = df[['client_id','contact_duration', 'number_contacts',
                 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
economics = df[['client_id','cons_price_idx',
                   'euribor_three_months']]

## 1. Cleaning Client dataset

In [161]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


In [162]:
client['credit_default'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [163]:
# Replace '.' with '_' in 'job' and 'education' columns
client.loc[:, 'job'] = client['job'].str.replace(".", "_", regex=False)
client.loc[:, 'education'] = client['education'].str.replace(".", "_", regex=False)
client.loc[:, 'education'] = client['education'].replace("unknown", np.nan)
client.loc[:, 'credit_default'] = client['credit_default'].replace("unknown", "no")

In [164]:
# Convert 'credit_default' and 'mortgage' columns to boolean (1 if 'yes', otherwise 0)
client.loc[:, 'credit_default'] = client['credit_default'].apply(lambda x: 1 if x == 'yes' else 0).astype(bool)
client.loc[:, 'mortgage'] = client['mortgage'].apply(lambda x: 1 if x == 'yes' else 0).astype(bool)

In [165]:
client['credit_default'].unique()

array([False, True], dtype=object)

In [166]:
client

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,False,True
41184,41184,46,blue-collar,married,professional_course,False,False
41185,41185,56,retired,married,university_degree,False,True
41186,41186,44,technician,married,professional_course,False,False


In [167]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


## 2. Cleaning Campaing dataset

In [168]:
# Display information about the campaign DataFrame
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   contact_duration            41188 non-null  int64 
 2   number_contacts             41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
dtypes: int64(4), object(2)
memory usage: 1.9+ MB


In [169]:
campaign['previous_outcome'].unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [170]:
# Convert 'previous_campaign_contacts' to boolean (1 if 'success', otherwise 0)
campaign.loc[:, 'previous_outcome'] = campaign['previous_outcome'].apply(lambda x: 1 if x == "success" else 0).astype(bool)

In [171]:
# Convert 'campaign_outcome' to boolean (1 if 'yes', otherwise 0)
campaign.loc[:, 'campaign_outcome'] = campaign['campaign_outcome'].apply(lambda x: 1 if x == 'yes' else 0).astype(bool)

In [172]:
# Create a new column 'year' with a fixed value
df['year'] = 2022

In [173]:
# Combine 'day', 'month', and 'year' columns to create a new 'last_contact_date' column in the campaign DataFrame
campaign.loc[:, 'last_contact_date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'] + '-' + df['day'].astype(str), format='%Y-%b-%d')

<ipython-input-173-e104755be4a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.loc[:, 'last_contact_date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'] + '-' + df['day'].astype(str), format='%Y-%b-%d')


In [174]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   contact_duration            41188 non-null  int64         
 2   number_contacts             41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  object        
 5   campaign_outcome            41188 non-null  object        
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 2.2+ MB


In [175]:
# Display DataFrame
campaign

,client_id,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,1,0,False,False,2022-05-13
1,1,149,1,0,False,False,2022-05-19
2,2,226,1,0,False,False,2022-05-23
3,3,151,1,0,False,False,2022-05-27
4,4,307,1,0,False,False,2022-05-03
...,...,...,...,...,...,...,...
41183,41183,334,1,0,False,True,2022-11-30
41184,41184,383,1,0,False,False,2022-11-06
41185,41185,189,2,0,False,False,2022-11-24
41186,41186,442,1,0,False,True,2022-11-17


## 3. Cleaning Economics dataset

In [176]:
# Display information about the economics DataFrame
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [177]:
# Display the first few rows of the economics DataFrame
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [178]:
# Save the three DataFrames to CSV files
client.to_csv('client.csv', index=False, sep=',')
campaign.to_csv('campaign.csv', index=False, sep=',')
economics.to_csv('economics.csv', index=False, sep=',')